## 1. Lab setup
- Download the [Network Automation appliance](https://www.gns3.com/marketplace/appliance/network-automation) from GNS3 marketplace.
- Create the simple topology as the figure below in GSN3.

## 2. Configure the router
We need to configure the router ip address to 192.167.122.71 since
the network automation container in GNS3 uses the subnet
192.168.122.0/24 by default. Check the ping to the subnet.
```bash
en
conf t
host R1
int g0/0
ip address 192.168.122.71 255.255.255.0
no shut
end
sh ip int brief
ping 192.168.122.1
```
We need to configure the default route, name-server of google for
Internet connection through the NAT cloud.
```bash
conf t
ip route 0.0.0.0 0.0.0.0 192.168.122.1 # configure the default route
ip domain-lookup
ip name-server 8.8.8.8
end
ping google.com
```
We need to enable password for telnet connection to the router. We
set `transport input all` so that all types of connections are
permitted includes telnet, ssh.
```bash
conf t
enable password cisco
username doanh password cisco
line vty 0 4
login local
transport input all
end
telnet 192.168.122.71 # check if telnet works
wr
```
## 3. Telnet to the router from network automation container
The container has already installed python2, python3 and some other
tools for network automation such as: netmiko, napalm, pyntc, and
Ansible.

- Configure the IP address of the container. Stop and restart the
container in GNS3 for the new IP address.
```bash
nano /etc/network/interfaces
auto eth0
iface eth0 inet dhcp
```

- Create the [python script](https://docs.python.org/2/library/telnetlib.html#telnet-example)
for telnet connection.
  - change the HOST to 192.168.122.71.
  - change to `tn.read_until("Username: ")` to fit with the cisco
  router telnet session.
  - Using `tn.write` to add the loopback 0 interface and configure
  ospf.

```python
import getpass
import telnetlib

HOST = "192.168.122.71"
user = raw_input("Enter your telnet username: ")
password = getpass.getpass()

tn = telnetlib.Telnet(HOST)

tn.read_until("Username: ")
tn.write(user + "\n")
if password:
    tn.read_until("Password: ")
    tn.write(password + "\n")

tn.write("enable\n")
tn.write("cisco\n")
tn.write("conf t\n")
tn.write("int loop 0\n")
tn.write("ip address 1.1.1.1 255.255.255.255\n")
tn.write("router ospf 1\n")
tn.write("network 0.0.0.0 255.255.255.255 area 0\n")
tn.write("end\n")
tn.write("exit\n")

print(tn.read_all())
```

- Debug on the router: Use `un all` to stop debugging.

```bash
debug telnet
term mon
```
- Run the python script on the container: `python FirstScript.py`
- Run `sh ip protocols` on the router to check OSPF is set or not.

## 4. Switch automation
- On switch S1, we see that only the default vlan is configured.
```bash
sh vlan
sh vlan brief
conf t
host S1
enable password cisco
username doanh password cisco
line vty 0 4
login local
transport input all
end
sh ip int brief
```
- Configure the ip address on vlan 1:
```bash
conf t
int vlan 1
no shut
ip address 192.168.122.72 255.255.255.0
end
wr
```
- Copy the FirstScript.py to PythonScript2.py

```python
import getpass
import telnetlib

HOST = "192.168.122.71"
user = raw_input("Enter your telnet username: ")
password = getpass.getpass()

tn = telnetlib.Telnet(HOST)

tn.read_until("Username: ")
tn.write(user + "\n")
if password:
    tn.read_until("Password: ")
    tn.write(password + "\n")

tn.write("enable\n")
tn.write("cisco\n")
tn.write("conf t\n")
tn.write("vlan 2\n")
tn.write("name Python_VLAN2\n")
tn.write("vlan 3\n")
tn.write("name Python_VLAN3\n")
tn.write("end\n")
tn.write("exit\n")

print(tn.read_all())
```
## 5. Python3 automation
```python
import getpass
import telnetlib

HOST = "192.168.122.71"
user = input("Enter your telnet username: ")
password = getpass.getpass()

tn = telnetlib.Telnet(HOST)

tn.read_until(b"Username: ")
tn.write(user.encode('ascii') + b"\n")
if password:
    tn.read_until(b"Password: ")
    tn.write(password.encode('ascii') + b"\n")

tn.write(b"enable\n")
tn.write(b"cisco\n")
tn.write(b"conf t\n")
tn.write(b"int loop 0\n")
tn.write(b"ip address 1.1.1.1 255.255.255.255\n")
tn.write(b"router ospf 1\n")
tn.write(b"network 0.0.0.0 255.255.255.255 area 0\n")
tn.write(b"end\n")
tn.write(b"exit\n")

print(tn.read_all().decode('ascii'))
```


